In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

plt.gray()

Using TensorFlow backend.


In [2]:
# function to display progress

def print_progress(org_percentage):
    percentage = int(org_percentage / 2)
    if percentage < 1:
        print("[{}]".format(" " * 50), end="\r")
    elif 0 < percentage < 50:
        print("[{}] {}/100".format("=" * (percentage - 1) + ">" + " " * (50 - percentage), org_percentage), end="\r")
    else:
        print("[{}] 100/100".format("=" * 50))

# A

## Lösungsansatz
Um festzustellen welches Bild zu welchem Modell gehört, schauen wir uns an wie gut die beiden Modelle im Vergleich bei
jedem einzelnem Bild performen, denn wenn das Modell eins der Bilder zum trainieren verwendet hat ist es wahrscheinlich,
dass es das Bild richtig mit einer hohen Confidence vorhersagt.

Um einen Vergleichswert zu haben laden wir das MNIST Datenset und suchen die original Labels.

## Bilder, Modelle und MNIST Set laden

In [3]:
images = np.load("samples.npy")

model_one = keras.models.load_model("model1.hdf5")
model_two = keras.models.load_model("model2.hdf5")

# train/test ist egal
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
X = np.concatenate([X_train, X_test])
Y = np.concatenate([Y_train, Y_test])

## Original Labels aus dem MNIST Datenset extrahieren
Zuerst formatieren wir die Bilder aus der Datei etwas, damit wir sie mit den Bildern aus dem MNIST Datenset vergleichen
können:
* entfernen der letzten Dimension
* mit 255 multiplizieren und zu `int` konvertieren

Danach vergleichen wir den Array aus Pixeln mit jedem Bild aus dem MNIST-Datenset.

In [4]:
images_flat = (images[:, :, :, 0] * 255).astype(int)

data = []

print("Labeling Images...")
progress = 0

for image in images_flat:
    print_progress(progress)
    found = False
    idx = 0
    for x in X:
        if np.array_equal(image, x):
            found = True
            data.append({
                "label": Y[idx],
                "image": image
            })
            continue
        idx += 1
    if not found:
        raise ValueError("Could not determine Label of Image")
    progress += 1

print_progress(progress)
print("Done!")

# 
for i in range(len(data)):
    data[i]["id"] = i

Labeling Images...
[==================================================] 100/100
Done!


## Beide Modelle alle Bilder vorhersagen lassen

In [5]:
res_model_one = model_one.predict(images)
res_model_two = model_two.predict(images)

## Validierung der Vorhersagen
Um eine erste Einteilung vorzunehmen, wenden wir folgende Regeln an:
* Wenn nur eines von beiden Modellen die Zahl richtig vorhergesagt markieren wir das Bild mit `1`/`2`für das jeweilige
Modell und speichern den Score des Modells
* Wenn beide das Bild richtig vorhergesagt haben wird das Bild mit `b` für both gekennzeichnet und die scores werden
gespeichert
  * Falls beide gleich gut (auf vier nachkommastellen) sind wird das Bild mit `e` für equal markiert
* Wenn beide Modelle eine falsche Vorhersage gemacht haben wird das Bild mit `n` für none markiert

In [6]:
"""
Data-Structure:
{
    "key": indicates which model got it right (b: both, n: none, 1, 2)
    "model": indicates which model performed best (e: equal, 1, 2)
    "label": contains the label extracted from the MNIST dataset
    "image": contains the image data
    "model_one": score for model one -> None if the prediction was wrong
    "model_two": score for model two -> None if the prediction was wrong
    "result": contains the final result -> will be modified later on
}
"""

#result = []

for idx in range(len(images)):
    # Ergebnisse der Größe nach aufsteigend sortieren und den letzten Index nehmen 
    one_val_idx = np.argsort(res_model_one[idx])[len(res_model_one[idx]) - 1]
    two_val_idx = np.argsort(res_model_two[idx])[len(res_model_two[idx]) - 1]
    data_point = {
        "key": "n",
        "model": None,
        "model_one": None,
        "model_two": None,
    }
    if one_val_idx != data[idx]["label"] and two_val_idx != data[idx]["label"]:
        data_point["key"] = "n"
        #result.append("n")
    elif one_val_idx == data[idx]["label"] != two_val_idx:
        data_point["key"] = 1
        data_point["result"] = 1
        data_point["model"] = 1
        data_point["model_one"] = res_model_one[idx][one_val_idx]
        #result.append(1)
    elif two_val_idx == data[idx]["label"] != one_val_idx:
        data_point["key"] = 2
        data_point["result"] = 2
        data_point["model_two"] = res_model_two[idx][two_val_idx]
        #result.append(2)
    elif res_model_one[idx][one_val_idx] == res_model_two[idx][two_val_idx]:
        data_point["key"] = "b"
        data_point["model"] = "e"
        data_point["model_one"] = res_model_one[idx][one_val_idx]
        data_point["model_two"] = res_model_two[idx][two_val_idx]
        #result.append("e")
    else:
        data_point["key"] = "b"
        data_point["model"] = 1 if res_model_one[idx][one_val_idx] > res_model_two[idx][two_val_idx] else 2
        data_point["result"] = 1 if res_model_one[idx][one_val_idx] > res_model_two[idx][two_val_idx] else 2
        data_point["model_one"] = res_model_one[idx][one_val_idx]
        data_point["model_two"] = res_model_two[idx][two_val_idx]
        #result.append(1 if res_model_one[idx][one_val_idx] > res_model_two[idx][two_val_idx] else 2)
    data[idx] = {**data[idx], **data_point}

### Ergebnis

In [7]:
one = 0
two = 0
non = 0
bth = 0

for el in data:
    if el["key"] == 1:
        one += 1
    elif el["key"] == 2:
        two += 1
    elif el["key"] == "n":
        non += 1
    elif el["key"] == "b":
        bth += 1
    else:
        raise ValueError("Unkown key {}".format(el["key"]))

print("None: {}\nOne: {}\nTwo: {}\nBoth: {}".format(non, one, two, bth))

None: 0
One: 20
Two: 0
Both: 80


Aufteilung auf Modell eins und zwei:

In [8]:
one = 0
two = 0
equ = 0

for el in data:
    if el["key"] == 1:
        one += 1
    elif el["key"] == 2:
        two += 1
    elif el["key"] == "b":
        if el["model"] == 1:
            one += 1
        elif el["model"] == 2:
            two += 1
        elif el["model"] == "e":
            equ += 1
        else:
            raise ValueError("Error")
    else:
        raise ValueError("Error")

print("One: {}\nTwo: {}".format(one, two))

One: 66
Two: 34


## Auswertung 
Wie wir sehen können haben wir keine Bilder die von keinem Modell richtig vorhergesagt wurden und auch keine Bilder bei
denen beide Modell gleich gut performed haben. Allerdings hat Modell eins wesentlich mehr Bilder zugewiesen bekommen
als Modell 2.  
Um die Aufteilung 50/50 zu erhalten, schauen wir jetzt welche Bilder von Modell eins zu Modell zwei umschichten können.

Kriterien:
1. Beide Modelle müssen das Bild richtig vorhergesagt haben
*  Modell 1 muss das Bild besser vorhergesagt haben als Modell 2
*  Modell 2 muss das Bild mit möglichst viel Confidence vorhergesagt haben
*  Modell 1 muss das Bild mit möglichst wenig Confidence vorhergesagt haben

Um die optimalen Bilder zu finden bilden wir für jedes Bild, das Kriterien 1 & 2 erfüllt, einen Score aus Kriterium
3 & 4:  
`Score = 1 - Model One Confidence + Model Two Confidence`  
_Je niedriger die Confidence von Modell 1 desto höher der Score und je höher die Confidence von Modell 2 desto höher der
Score._  
Danach wählen wir die 16 Bilder mit dem höchsten Score aus und ordnen sie dem zweiten Modell zu.

In [9]:
for (i, el) in enumerate(data):
    if el["key"] != "b" or (el["key"] == "b" and el["model_one"] < el["model_two"]):
        continue
    data[i]["score"] = 1 - el["model_one"] + el["model_two"]

data.sort(key=lambda el: el["score"] if "score" in el.keys() else 0, reverse=True)

for i in range(16):
    data[i]["result"] = 2

# sort back to normal order for use in next task and to show result
data.sort(key=lambda el: el["id"])

print([el["result"] for el in data])

[1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1]


# B

In [10]:
model_one_labels = []
model_two_labels = []

for el in data:
    if el["model"] == 1:
        model_one_labels.append(el["label"])
    elif el["result"] == 2:
        model_two_labels.append(el["label"])

def print_results(model):
    (keys, values) = model
    for i in range(len(keys)):
        print("\tLabel: {} Count: {}".format(keys[i], values[i]))

print("Model One:")
print_results(np.unique(model_one_labels, return_counts=True))
print("Model Two:")
print_results(np.unique(model_two_labels, return_counts=True))

Model One:
	Label: 0 Count: 8
	Label: 1 Count: 6
	Label: 3 Count: 8
	Label: 4 Count: 10
	Label: 6 Count: 10
	Label: 7 Count: 16
	Label: 9 Count: 8
Model Two:
	Label: 0 Count: 2
	Label: 1 Count: 14
	Label: 3 Count: 2
	Label: 4 Count: 10
	Label: 7 Count: 4
	Label: 9 Count: 2


## Auswertung
Wie man sehr deutlich sehen kann hat Modell eins wesentlich häufiger Bilder von siebenern richtig/besser erkannt und
Modell 2 hat bei einsern wesentlich besser performed. Bei der 4 haben beide Modell ziemlich genau gleich gut
gearbeitet.

Vermutung:
* vierer waren ungefähr gleichmäßig verteilt
* Modell 1:
  * hauptsächlich 6, 7, 9
  * generell ausgewogener trainiert worden
* Modell 2:
  * hauptsächlich 1
  * keine 6 sehr wenig 0, 3, 9
  * auf sehr unausgewogenem Set trainiert worden

Außerdem kann man bei der Auswertung wieveiele Ziffern nur von 1/2 oder beiden richtig vorhergesagt wurden sehen, dass
Modell 2 keine einzige Ziffer alleine richtig vorhergesagt hat

# C

### A
Die Datensets auszugleichen ist nicht einfach zuerst habe ich probiert Bilder einzeln anzuschauen und so zuzuordnen,
allerdings ist das nahezu unmöglich und auch nur bedingt begründbar, der Score war dann die einfachste und logischte
Lösung  
### B
